In [1]:
import sys
import os
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_meinhold')
sys.path.append('../utils_meinhold')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca')
sys.path.append('../utils_zonca')
#sys.path.append('D:\\software_git_repos\\cofe-python-analysis-tools\\utils_zonca\\pointing')
sys.path.append('../utils_zonca/pointing')
#sys.path.append('D:\\software_git_repos\\greenpol')
sys.path.append('../')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\')
sys.path.append('../telescope_control')
#sys.path.append('D:\\software_git_repos\\greenpol\\telescope_control\\VtoT\\')
sys.path.append('../VtoT')
import realtime_gp as rt
import numpy as np
import datetime  
import h5py
import pandas as pd
from pointingtools import compute_parallactic_angle, altaz2ha 
from planets import getlocation
import warnings
from astropy.coordinates import AltAz, Angle, EarthLocation, ICRS, SkyCoord, frame_transform_graph
from astropy import units as u
import ephem
import matplotlib.pyplot as plt
import time
from datetime import datetime
from planets import getlocation, getpointing

import Tkinter,tkFileDialog

from Tkinter import *
import ttk

import pickle
import glob

import cPickle
from scipy.signal import savgol_filter

%pylab


Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


c:\users\labuser\anaconda3\envs\py27\lib\site-packages\IPython\core\magics\pylab.py:161: UserWarning: pylab import has clobbered these variables: ['Widget', 'Text', 'Button', 'datetime']
`%matplotlib` prevents importing * from pylab and numpy
  "\n`%matplotlib` prevents importing * from pylab and numpy"


In [2]:
%matplotlib auto

Using matplotlib backend: TkAgg


In [3]:
def get_pointing_files(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/pointing_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    return filelist

In [4]:
def get_file_times(fld):
    startfile = fld[0][:43]+fld[0][49:-2]+'dat'
    endfile = fld[-1][:43]+fld[-1][49:-2]+'dat'
    
    #starttime = os.path.getctime(startfile)
    starttime= os.stat(startfile).st_mtime + 7*3600
    starttime = datetime.datetime.fromtimestamp(starttime)
    
    #endtime = os.path.getctime(endfile)
    endtime= os.stat(endfile).st_mtime + 7*3600
    endtime = datetime.datetime.fromtimestamp(endtime)
    
    return starttime, endtime

In [39]:
def fileStruct(n_array, chan, starttime, endtime, cleaned=False):

    fpath = "C:/software_git_repos/polaris/polaris_data/level1"
    os.chdir(fpath)
    yrmoday = starttime.strftime('%Y%m%d')
    path = '-'.join((starttime.strftime('%H_%M_%S'), endtime.strftime('%H_%M_%S')))
    path = '-'.join((yrmoday, path))
    path = '-'.join((chan, path))
    if cleaned:
        path = path+'_cleaned'
    print 'start time: ', starttime
    print 'end time: ', endtime
    print 'elapsed time: ', (endtime - starttime).total_seconds(), 'sec'
    if not os.path.exists(yrmoday):#this is the first file being created for that time
        os.makedirs(yrmoday)
        #set index to 0

    path = '/'.join((yrmoday,path))
    path = '.'.join((path,"h5"))
    with h5py.File(str(path).replace("pkl","h5"), mode="w") as f:
        f.create_dataset("data", data=n_array.to_records(index=False))

In [6]:
def round_fraction(number, res):
    amount = int(number/res)*res
    remainder = number - amount
    return amount if remainder < res/2. else amount+res

In [7]:
def read_some_data(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    dlist=[]
    for f in filelist:
        hf=h5py.File(f)
        dlist.append(hf['demod_data'])
    d=np.concatenate(dlist)
    hf.close() 

    datadict=d
        
    return datadict, filelist

In [8]:
def convert_gpstime(starttime, gpstime, ltoffset=0, bits_to_ms=2**24, format='seconds', ttype = 'utc', singletime=False):

    #function to convert gpstime to UTC time or local time

    #universal time day
    ltoffset = ltoffset*60*60
    utcday = starttime + datetime.timedelta(0, ltoffset)

    #days since last sunday
    idx = (utcday.weekday() + 1) % 7

    #date of previous sunday in universal coord
    sunday = utcday - datetime.timedelta(idx) 

    syear = int(str(sunday)[:4])
    smonth = int(str(sunday)[5:7])
    sday = int(str(sunday)[8:10])
    
    sunday = datetime.datetime(syear, smonth, sday, 0, 0, 0)

    #seconds since last sunday
    sundaysec = (utcday - sunday).total_seconds()

    #convert to seconds
    bits_to_sec = bits_to_ms/1000
    
    #number of wraps so far
    numwraps = int(sundaysec/bits_to_sec)
    
    #time of last wrap
    gpsstarttime = sunday + datetime.timedelta(0, numwraps*bits_to_sec)
    
    #convert gps starttime to timestamp
    #gpsstarttime = (gpsstarttime-datetime.datetime(1970,1,1)).total_seconds()
    gpsstarttime = time.mktime(gpsstarttime.timetuple())
    
    if singletime == False:
        #find all wrap points in current data set
        gpsdiff = np.diff(gpstime)
        iwrap = np.where(gpsdiff < -2**24/1000/2)[0]
        #iwrap = np.where(abs(gpsdiff) > 2**24/1000/2)[0]

        #unwrap gpstime
        for w in iwrap:
            gpstime[w+1:] = gpstime[w+1:] + gpstime[w]

    if ttype == 'utc':
        ltoffset = 0
    #gpstime gives seconds since starting point
    dtime = gpsstarttime + gpstime - ltoffset

        
    if format == 'datetime':
        if singletime == False:
            t = []
            for i in range(len(dtime)):
                t.append(datetime.datetime.fromtimestamp(dtime[i]))
        else:
            t = datetime.datetime.fromtimestamp(dtime)
        
        return t, dtime
    
    else:
        return dtime

    '''
    dtime = []
     
    for i in range(len(gpstime)):
    #for i in range(0,1):
        
        if i>0:
            if abs(gpstime[i] - gpstime[i-1]) > 2**24/1000/2:
                print 'it got here'
                gpstime[i:] = gpstime[i:] + gpstime[i-1]
                
        #gpstime gives seconds since starting point
        t = gpsstarttime + datetime.timedelta(0,gpstime[i]) - datetime.timedelta(0, ltoffset)

        if format == 'seconds':
            timestamp = (t-datetime.datetime(1970,1,1)).total_seconds()
            dtime.append(timestamp)
        else:
            dtime.append(t)
          
    return dtime
    '''

In [9]:
def flagging(channel, samples_per_rev, hkflags, baseline=False):
    '''
    Read time stream and apply flagging
    '''

    tod_len = np.size(channel)
    
    flags = np.ones(tod_len, dtype=bool)

    if baseline == False:
        baseline = 4*np.std(channel)
    else:
        baseline = baseline   


    for i in range(int(tod_len/samples_per_rev)):
        i_low  = i*samples_per_rev 
        i_high = i*samples_per_rev + samples_per_rev-1

        sigma  = np.std(channel[i_low:i_high+1])

        threshold = baseline/sigma 
        #print baseline

        median = np.median(channel[i_low:i_high+1])

        mask1  = np.array(np.where(channel[i_low:i_high+1]>median+threshold*sigma)) + i_low
        mask2  = np.array(np.where(channel[i_low:i_high+1]<median-threshold*sigma)) + i_low

        flags[mask1] = False
        flags[mask2] = False
        
    #mask3 = np.where(hkflags != 1011)[0]
    #flags[mask3] = False
        
    return flags 

In [10]:
def cleanish_data(data1, hkflags, samples_per_rev, data2=[]):
    
    dsmoothed = savgol_filter(data1, 51, 3)
    d1 = np.diff(dsmoothed)
    
    flags = flagging(data1, samples_per_rev, hkflags)
    flags2 = flagging(d1, samples_per_rev, hkflags)
    
    if len(data2) != 0:
        dcleaned = data2.copy()
    else:
        dcleaned = data1.copy()
    dcleaned[np.where(flags==False)] = np.nan
    dcleaned[np.where(flags2==False)] = np.nan
    dcleaned[np.where(hkflags != 1011)] = np.nan
    
    return dcleaned

In [11]:
def smoothTriangle(data, degree):
    triangle=np.concatenate((np.arange(degree + 1), np.arange(degree)[::-1])) # up then down
    smoothed=[]

    for i in range(degree, len(data) - degree * 2):
        point=data[i:i + len(triangle)] * triangle
        smoothed.append(np.sum(point)/np.sum(triangle))
    # Handle boundaries
    smoothed=[smoothed[0]]*int(degree + degree/2) + smoothed
    while len(smoothed) < len(data):
        smoothed.append(smoothed[-1])
    return smoothed

In [43]:
ddict={}

In [44]:
dd, fld = read_some_data()

In [45]:
flp = get_pointing_files()
pp = rt.get_h5_pointing(flp)

In [46]:
combined = rt.combine_cofe_h5_pointing(dd,pp)

In [17]:
t = (combined['gpstime'] - combined['gpstime'][0])/1000./3600.

In [47]:
starttime, endtime = get_file_times(fld)

print starttime
print endtime

2019-10-25 07:43:20.641829
2019-10-25 15:11:19.873233


In [60]:
chan = 'H1HiAC'
var = 'T'
LOCATION = 'Sedgwick'
spr = 256

#gain in kelvin per volt
changain = {'ch0':-4.89, 'ch1':-2.85*30., 'ch4':-9.47, 'ch8':-11.1}

In [61]:
az = combined['az']
el = combined['el']

gpstime = dd['rev']/1000
ltoffset = 0
dtime = convert_gpstime(starttime, gpstime, ltoffset, format = 'datetime', ttype = 'ltc')[1]

In [62]:
'''
#t = (combined['gpstime'] - combined['gpstime'][0])/1000./3600.
t = combined['gpstime']/1000
sph = 60.
sps = sph/3600.
sunpoints = np.arange(t[0], t[-1] + (1/sps), 1/sps)

dt = convert_gpstime(starttime, sunpoints, ltoffset, format = 'datetime', ttype = 'ltc')[1]

saz = []
sel = []
for d in dt:
    sa, se = getpointing('Greenland', 'Sun', d)
    saz.append(sa)
    sel.append(se)
    
saz = np.asarray(saz)
sel = np.asarray(sel)
'''

        Use `lon` instead. [astropy.utils.decorators]
        Use `lat` instead. [astropy.utils.decorators]


In [ ]:
#saz = np.interp(combined['gpstime'], sunpoints, saz)

In [22]:
saz, sel = getpointing('Greenland', 'Sun', dtime[0])
print saz, sel

saz = []
sel = []
for d in dtime:
    sa, se = getpointing('Greenland', 'Sun', d)
    saz.append(sa)
    sel.append(se)
    
saz = np.asarray(saz)
sel = np.asarray(sel)


        Use `lon` instead. [astropy.utils.decorators]
        Use `lat` instead. [astropy.utils.decorators]


231.597027778 17.60175


In [46]:
def suncut(cut, saz, sel, az, el, data):
    angdiff = np.degrees(np.arccos(np.sin(np.radians(az))*np.sin(np.radians(saz)) 
                    + np.cos(np.radians(az))*np.cos(np.radians(saz))*np.cos(np.radians(el) - np.radians(sel))))
    
    dd = data*1
    icut = np.where(abs(angdiff) > cut)[0]
    dd[icut] = np.nan
    
    return dd

In [67]:
dd = suncut(5., saz, sel, az, el, az)

In [70]:
figure()
plot(sel)

In [68]:
figure()
plot(az, label='angular distance')
plot(dd, label='cut angular distance')
#plot(data+45, label='data')
legend()

In [63]:
data = combined['sci_data'][rt.nametochan(chan)][var]*changain[rt.nametochan(chan)]
Q = combined['sci_data'][rt.nametochan(chan)]['Q']*changain[rt.nametochan(chan)]
hkflags = combined['flag']

In [64]:
#offset between local time and utctime in hours
ltoffset = 0

#seconds since last sunday utc
gpstime = dd['rev']/1000#(combined['gpstime'])/1000

In [65]:
figure()
plot(dd['rev'])

In [54]:
gpstime = dd['rev']
diff = np.diff(gpstime)
ijump = np.where(diff > 1e4)[0][0]
dmax = diff.max()
dmin = diff.min()
print gpstime[ijump:ijump+2]
gpstime[ijump+1:] -= (dmax - dmin)
print gpstime[ijump:ijump+2]
print diff

[33554420. 50331654.]
[33554420. 33552992.]
[30. 28. 30. ... 30. 28. 30.]


In [66]:
gpstime = dd['rev']/1000#(combined['gpstime'])/1000
dtime = convert_gpstime(starttime, gpstime, ltoffset, format = 'datetime', ttype = 'ltc')[0]
tdiffe = (dtime[-1] - endtime).total_seconds()
tdiffs = (dtime[0] - starttime).total_seconds()
print 'file start time: ', starttime
print 'gps converted start time: ', dtime[0]
print 'amount off in seconds: ', tdiffs
print 'file end time: ', endtime
print 'gps converted end time: ', dtime[-1]
print 'amount off in seconds: ', tdiffe

file start time:  2019-10-25 07:43:20.641829
gps converted start time:  2019-10-25 07:46:19.739000
amount off in seconds:  179.097171
file end time:  2019-10-25 15:11:19.873233
gps converted end time:  2019-10-25 15:14:45.982000
amount off in seconds:  206.108767


In [67]:
#gpstime = dd['rev']/1000#(combined['gpstime'])/1000
dtime, utime = convert_gpstime(starttime, gpstime, ltoffset, format = 'datetime')

In [68]:
from astropy.utils.iers import conf
conf.auto_max_age = None

In [69]:
az = combined['az']
el = combined['el'] 

location = getlocation(LOCATION)

t1 = time.time()
#create ra dec sky object
azel = SkyCoord(az = az, alt = el, obstime = dtime, location = location, frame = 'altaz', unit='deg')
t2 = time.time()
print t2-t1
with warnings.catch_warnings():
    warnings.simplefilter("ignore")
    #convert from ra dec to az/el for pointing
    radec = azel.icrs
print time.time()-t2
ra = radec.ra.rad
dec = radec.dec.rad

3.20399999619
224.978000164


In [70]:
AZ = np.radians(az)
EL = np.radians(el)

In [71]:
lat = location.latitude.rad
ha = altaz2ha(EL, AZ, lat)
psi = compute_parallactic_angle(ha, lat, dec)

        Use `lat` instead. [astropy.utils.decorators]


In [38]:
#need to figure out units on TIME in andreas test file

In [72]:
spr = 256#*30

In [75]:
#uncleaned data
#cleaned data
Tdata = combined['sci_data'][rt.nametochan(chan)]['T']*changain[rt.nametochan(chan)]
h5data = pd.DataFrame({"TEMP" : Tdata})

h5data["PHI"] = ra

h5data["THETA"] = np.pi/2 - dec

h5data["PSI"] = psi

h5data["FLAG"] = np.zeros(len(psi))

Qdata = combined['sci_data'][rt.nametochan(chan)]['Q']*changain[rt.nametochan(chan)]
h5data["Q"] = Qdata

Udata = combined['sci_data'][rt.nametochan(chan)]['U']*changain[rt.nametochan(chan)]
h5data["U"] = Udata

In [77]:
#cleaned data
Tdata = combined['sci_data'][rt.nametochan(chan)]['T']*changain[rt.nametochan(chan)]
Tcleaned = cleanish_data(Tdata, hkflags, spr)
Tfinal = Tcleaned[np.logical_not(np.isnan(Tcleaned))]
h5data = pd.DataFrame({"TEMP" : Tfinal})

raclean = ra[np.logical_not(np.isnan(Tcleaned))]
h5data["PHI"] = raclean

decclean = dec[np.logical_not(np.isnan(Tcleaned))]
h5data["THETA"] = np.pi/2 - decclean

psiclean = psi[np.logical_not(np.isnan(Tcleaned))]
h5data["PSI"] = psiclean

h5data["FLAG"] = np.zeros(len(psiclean))

Qdata = combined['sci_data'][rt.nametochan(chan)]['Q']*changain[rt.nametochan(chan)]
Qcleaned = cleanish_data(Tdata, hkflags, spr, Qdata)
Qfinal = Qcleaned[np.logical_not(np.isnan(Qcleaned))]
h5data["Q"] = Qfinal

Udata = combined['sci_data'][rt.nametochan(chan)]['U']*changain[rt.nametochan(chan)]
Ucleaned = cleanish_data(Tdata, hkflags, spr, Udata)
Ufinal = Ucleaned[np.logical_not(np.isnan(Ucleaned))]
h5data["U"] = Ufinal

In [83]:
print dtime[0]

2019-10-25 07:46:19.739000


In [84]:
figure()
plot(Qdata, label='Uncleaned')
plot(Qcleaned, label='Cleaned')
#plot(dd, label='sun cut: $\Delta \Theta > %s^{\degree}$' % cut)
ylabel('Volts')
xlabel('Samples')
title('Sedgwick 10/25/19 el = $70^{\degree}$, %s %s' % (chan, 'Q'))
legend()

In [78]:
fileStruct(h5data, chan, starttime, endtime, cleaned=True)

start time:  2019-10-25 07:43:20.641829
end time:  2019-10-25 15:11:19.873233
elapsed time:  26879.231404 sec


In [688]:
az = combined['az']
el = combined['el'] 

In [728]:
dd = combined['sci_data'][rt.nametochan(chan)]['Q']*changain[rt.nametochan(chan)]
chan = 'H1HiAC'
var = 'Q'
res = 1.0
plotnow_azelsig(h5data["TEMP"], np.degrees(h5data["PHI"]), np.degrees(h5data["THETA"]), rt.nametochan(chan), var, res, radec=True)

c:\users\labuser\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:73: RuntimeWarning: invalid value encountered in divide


In [127]:
tt = cleanish_data(Tdata, hkflags, 256*30, t)

In [59]:
cut = 5.
dd = suncut(cut, saz, sel, az, el, Qcleaned)

In [109]:
f1 = 'C:/software_git_repos/polaris/polaris_data/level1/20191025/ch0-07_43_20-15_11_19.h5'
f2 = 'C:/software_git_repos/polaris/polaris_data/level1/20191030/ch0-09_26_34-13_38_17.h5'
hf1 = h5py.File(f1)
hf2 = h5py.File(f2)

a = np.concatenate([hf1['data']['TIME'], hf2['data']['TIME']])

In [121]:
figure()
plot(hf1['data']['Q'])
plot(hf2['data']['Q'])

ValueError: Not a location (invalid object ID)

In [118]:
print h5data.keys()[0]
f1 = 'C:/software_git_repos/polaris/polaris_data/level1/20191025/ch0-07_43_20-15_11_19.h5'
f2 = 'C:/software_git_repos/polaris/polaris_data/level1/20191030/ch0-09_26_34-13_38_17.h5'
hf1 = h5py.File(f1)
hf2 = h5py.File(f2)

TIME = np.concatenate([hf1['data']['TIME'], hf2['data']['TIME']])
PHI = np.concatenate([hf1['data']['PHI'], hf2['data']['PHI']])
THETA = np.concatenate([hf1['data']['THETA'], hf2['data']['THETA']])
PSI = np.concatenate([hf1['data']['PSI'], hf2['data']['PSI']])
FLAG = np.concatenate([hf1['data']['FLAG'], hf2['data']['FLAG']])
TEMP = np.concatenate([hf1['data']['TEMP']*4.4/3.1, hf2['data']['TEMP']])
Q = np.concatenate([hf1['data']['Q']*4.4/3.1, hf2['data']['Q']])
U = np.concatenate([hf1['data']['U']*4.4/3.1, hf2['data']['U']])


h5data_2days=pd.DataFrame({"TIME" : TIME}) 
h5data_2days["PHI"] = PHI
h5data_2days["THETA"] = THETA
h5data_2days["PSI"] = PSI
h5data_2days["FLAG"] = FLAG
h5data_2days["TEMP"] = TEMP
h5data_2days["Q"] = Q
h5data_2days["U"] = U

hf1.close()
hf2.close()

TIME


In [80]:
print hf1

<Closed HDF5 file>


In [ ]:
def read_some_data(filelist=None):

    if filelist==None:
        root=Tkinter.Tk()
        filelist = list(tkFileDialog.askopenfilenames(\
        initialdir='D://software_git_repos/greenpol/telescope_control/data_aquisition/demod_data/',parent=root,title='Choose a set of files'))
        root.destroy()
    filelist.sort()
    
    dlist=[]
    for f in filelist:
        hf=h5py.File(f)
        dlist.append(hf['demod_data'])
    d=np.concatenate(dlist)
    hf.close() 

    datadict=d
        
    return datadict, filelist

In [51]:
figure()
plot(AZ)

In [716]:
def plotnow_azelsig(data, az, el, chan, var, res, minmax=None, radec=False, supply_index=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=get_h5_pointing(flp)
    ##dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=get_all_demodulated_data(fld_demod, fld)	
    #combined=combine_cofe_h5_pointing(dd,pp)

    #synchronized data az and el values
    az1, el1 = az.copy(), el.copy()
    data = data

    #convert to temp for cryo sensors
    if chan == 12:
        data = convert.convert(data, 'i')
    if chan == 13:
        data = convert.convert(data, 'e')
    if chan == 14:
        data = convert.convert(data, 'h')
    if chan == 15:
        data = convert.convert(data, 'l')

    steps = len(data)

    #set az/el resolution
    dx = res
    dy = res

    #set up bins/grid
    if radec:
        x, y = np.arange(0., 360.+dx, dx), np.arange(-90., 90. + dy, dy)
    else:
        x, y = np.arange(0., 360.+dx, dx), np.arange(0., 90. + dy, dy)
    AZ, EL = np.meshgrid(x, y)

    #small number for comparing floats
    epsilon = 1e-6

    #set up matrix for signal 
    z1 = np.zeros(len(x)*len(y))
    sig = np.reshape(z1, (len(y), len(x)))

    #set up matrix for keeping track of data points in single bin for averaging
    z2 = np.zeros(len(x)*len(y))
    count = np.reshape(z2, (len(y), len(x)))

    for i in range(steps):

        #round az/el points for comparison with grid	    
        el1[i] = rt.round_fraction(el1[i], dy)
        az1[i] = rt.round_fraction(az1[i], dx)  

        #find where data points belong in grid
        iel = np.where(abs(y - el1[i]) < epsilon)[0][0]
        iaz = np.where(abs(x - az1[i]) < epsilon)[0][0]
        
        #add 1 each time data point lands in same bin
        count[iel][iaz] += 1

        #add total number of data values in bin
        sig[iel][iaz] = sig[iel][iaz] + data[i]  
    

    #mask 0 count values so they dont show up in color plot
    count = ma.masked_where(count == 0.0, count)

    #take average of all data points in single bin
    sig = sig/count

    #change units on plot label
    if int(chan[2:]) < 12:
        unit = 'V'
    else:
        unit = 'K' 

    name = rt.chantoname(chan)

    figure()
    plt.pcolormesh(AZ, EL, sig)
    plt.colorbar(label = 'Signal, %s' % unit)
    if minmax != None:
        plt.clim(minmax[0],minmax[1])
    else:  
        plt.clim(data.min(),data.max())
    if radec == False:
        plt.axis([AZ.min(), AZ.max(), EL.min(), EL.max()])
        plt.ylabel('elevation (deg)')
        plt.xlabel('azimuth (deg)')
    else:
        plt.axis([0., 360., -90, 90.])
        plt.xlabel('ra (deg)')
        plt.ylabel('dec (deg)')
    plt.title('%s %s data binned to azimuth and elevation' % (name, var))
    plt.grid()
    plt.show()

In [142]:
figure()
plot(data)

In [139]:
chan = 'H1HiAC'
var = 'T'
az = combined['az']
data = combined['sci_data'][rt.nametochan(chan)][var]
plotnow_azrevsig(data, az, rt.nametochan(chan), var, res=1.0, minmax = [data.min()/20., data.max()/20.])

c:\users\labuser\anaconda3\envs\py27\lib\site-packages\ipykernel_launcher.py:78: RuntimeWarning: Mean of empty slice.


In [593]:
def plotnow_azrevsig(data, az, chan, var, res = 1.0, minmax=None,supply_index=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=get_h5_pointing(flp)
    #dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=get_all_demodulated_data(fld_demod, fld)
    #combined=combine_cofe_h5_pointing(dd,pp)

    #synchronized data and az values
    az1 = az
    data1 = data
    steps = len(data1)

    #convert to temp for cryo sensors
    if chan == 12:
        data1 = convert.convert(data1, 'i')
    if chan == 13:
        data1 = convert.convert(data1, 'e')
    if chan == 14:
        data1 = convert.convert(data1, 'h')
    if chan == 15:
        data1 = convert.convert(data1, 'l')

    #resolution
    dx = res
    dy = res

    #set up empty lists to append each revolution to
    data = []
    az = []
    iaz = [0]
    rev = 0

    #determine indices in azimuth/data array which correspond to a new revolution of the telescope
    for i in range(steps):
        #round values to resolution for comparison later
        az1[i] = round_fraction(az1[i], dx)
        if i > 0:
            if abs(az1[i] - az1[i-1]) >= 180.:
                iaz.append(i)
                rev += 1

    #append each revolution array to a list	    
    for j in range(rev):
        az.append(az1[iaz[j]:iaz[j+1]])
        data.append(data1[iaz[j]:iaz[j+1]])

    #append the last revolution
    data.append(data1[iaz[-1]:])
    az.append(az1[iaz[-1]:])
    rev += 1

    data = np.asarray(data)
    az = np.asarray(az)

    #create grid for plotting
    x, y = np.arange(0., 360.+dx, dx), np.arange(0., rev - 1 + dy, dy)
    AZ, REV = np.meshgrid(x, y)

    #set up empty array
    z = np.zeros(len(x)*len(y))
    sig = np.reshape(z, (len(y), len(x)))

    #small number for comparing floats
    epsilon = 1e-6

    #fill signal array with data points
    for r in range(rev):
        for a in range(len(x)):
            #find indices where combined azimuth data fits on x grid
            idx = np.where(abs(az[r] - x[a]) < epsilon)[0]
            #if idx length is 0 this will create a mask on that point, in idx len > 1, avg data points in the same bin
            sig[r][a] = data[r][idx].mean()

    #mask invalid values, i.e. where there are no data points
    sig = ma.masked_invalid(sig)

    #change units on plot label
    if int(chan[2:]) < 12:
        unit = 'V'
    else:
        unit = 'K' 

    name = rt.chantoname(chan)

    plt.pcolormesh(AZ, REV, sig)
    plt.colorbar(label = 'Signal, %s' % unit)

    if minmax != None:
        plt.clim(minmax[0],minmax[1])
    else:  
        plt.clim(data1.min(),data1.max())
    plt.axis([0., 360., 0., rev - 1])
    plt.ylabel('revolution #')
    plt.xlabel('azimuth (deg)')
    plt.title('%s %s data binned to azimuth and revolution #' % (name, var))
    plt.grid()
    plt.show()

In [626]:
def plotnow_azelsig2(data, az, el, chan, var, res, minmax=None, radec=False, supply_index=False):
    #flp=select_h5(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #fld_demod, fld =select_dat(fpath,yrmoday,st_hour,st_minute,ed_hour,ed_minute)
    #i=0
    #while len(flp)<3:
    #	i+=1
    #	flp=select_h5(fpath,yrmoday,st_hour,int(st_minute)-i,ed_hour,int(ed_minute)+i)

    #pp=get_h5_pointing(flp)
    ##dd=get_demodulated_data_from_list(fld,supply_index=supply_index)
    #dd=get_all_demodulated_data(fld_demod, fld)	
    #combined=combine_cofe_h5_pointing(dd,pp)

    #synchronized data az and el values
    az1, el1 = az, el
    data = data

    #convert to temp for cryo sensors
    if chan == 12:
        data = convert.convert(data, 'i')
    if chan == 13:
        data = convert.convert(data, 'e')
    if chan == 14:
        data = convert.convert(data, 'h')
    if chan == 15:
        data = convert.convert(data, 'l')

    steps = len(data)

    #set az/el resolution
    dx = res
    dy = res

    #set up bins/grid
    x, y = np.arange(0., 360.+dx, dx), np.arange(0., 90. + dy, dy)
    AZ, EL = np.meshgrid(x, y)

    #small number for comparing floats
    epsilon = 1e-6

    #set up matrix for signal 
    z1 = np.zeros(len(x)*len(y))
    sig = np.reshape(z1, (len(y), len(x)))

    #set up matrix for keeping track of data points in single bin for averaging
    z2 = np.zeros(len(x)*len(y))
    count = np.reshape(z2, (len(y), len(x)))

    for i in range(steps):

        #round az/el points for comparison with grid	    
        el1[i] = rt.round_fraction(el1[i], dy)
        az1[i] = rt.round_fraction(az1[i], dx)  

        #find where data points belong in grid
        iel = np.where(abs(y - el1[i]) < epsilon)[0][0]
        iaz = np.where(abs(x - az1[i]) < epsilon)[0][0]

        #add 1 each time data point lands in same bin
        count[iel][iaz] += 1

        #add total number of data values in bin
        sig[iel][iaz] = sig[iel][iaz] + data[i]  

    #mask 0 count values so they dont show up in color plot
    count = ma.masked_where(count == 0.0, count)

    #take average of all data points in single bin
    sig = sig/count

    #change units on plot label
    if int(chan[2:]) < 12:
        unit = 'V'
    else:
        unit = 'K' 

    name = rt.chantoname(chan)

    plt.pcolormesh(AZ, EL, sig)
    plt.colorbar(label = 'Signal, %s' % unit)
    if minmax != None:
        plt.clim(minmax[0],minmax[1])
    else:  
        plt.clim(data.min(),data.max())
    if radec == False:
        plt.axis([AZ.min(), AZ.max(), EL.min(), EL.max()])
        plt.ylabel('elevation (deg)')
        plt.xlabel('azimuth (deg)')
    else:
        plt.axis([0., 360., -90, 90.])
        plt.xlabel('ra (deg)')
        plt.ylabel('dec (deg)')
    plt.title('%s %s data binned to azimuth and elevation' % (name, var))
    plt.grid()
    plt.show()